### Converting between instruments using a Seq2Seq Model

In [2]:
import torch
import torchaudio
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import Dataset
import librosa
import numpy as np
import random

from pathlib import Path
from torch.utils.data import Dataset

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


Torchaudio processing methods

In [120]:
def normalized(tensor):
    # https://pytorch.org/tutorials/beginner/audio_preprocessing_tutorial.html
    centered = tensor - tensor.mean()
    normalized = tensor / tensor.abs().max()
    return normalized

_mu_encoder = torchaudio.transforms.MuLawEncoding()
_mu_decoder = torchaudio.transforms.MuLawDecoding()

def mu_law_encode(waveform):
    return _mu_encoder(normalized(waveform))

def mu_law_decode(waveform):
    return _mu_decoder(normalized(waveform))

def load_audio(path):
    """ Load .wav file to Mu law encoding tensor """
    waveform, sample_rate = torchaudio.load(path)
    return mu_law_encode(waveform), sample_rate

def save_audio(path, data, sample_rate):
    """ Save Mu law encoding tensor to .wav file """
    waveform = mu_law_decode(data)
    torchaudio.save(path, data, sample_rate)
    
SOS_token = -1
EOS_token = -2

Converting wav files to tensors and saving them

In [5]:
dataset = Path.cwd().joinpath("dataset")
train_wav = dataset.joinpath("train_wav")
train_tensors = dataset.joinpath("train_tensors")
test_wav = dataset.joinpath("test_wav")
test_tensors = dataset.joinpath("test_tensors")

for instrument in train_wav.iterdir():
    if (instrument.name != ".DS_Store"):
        for note in train_wav.joinpath(instrument.name).iterdir():
            if (note.name != ".DS_Store"):
                wav_tensor = load_audio(train_wav.joinpath(instrument.name, note.name))
                wav_tensor = (wav_tensor[0][0] + wav_tensor[0][1]) / 2
                torch.save(wav_tensor, train_tensors.joinpath(instrument.name, note.stem + ".pt"))
                
for instrument in test_wav.iterdir():
    if (instrument.name != ".DS_Store"):
        for note in test_wav.joinpath(instrument.name).iterdir():
            if (note.name != ".DS_Store"):
                wav_tensor = load_audio(test_wav.joinpath(instrument.name, note.name))
                wav_tensor = (wav_tensor[0][0] + wav_tensor[0][1]) / 2
                torch.save(wav_tensor, test_tensors.joinpath(instrument.name, note.stem + ".pt"))

Loading in the tensors and preparing them for the dataset

In [6]:
dataset = Path.cwd().joinpath("dataset")

#Change for model you are training
train_input = dataset.joinpath("train_tensors", "75")
train_output = dataset.joinpath("train_tensors", "81")
test_input = dataset.joinpath("test_tensors", "75")
test_output = dataset.joinpath("test_tensors", "81")

train_input_wav = []
train_output_wav = []
test_input_wav = []
test_output_wav = []

#Take out DS_Stores if you are using a Mac
for tensor in train_input.iterdir():
    if tensor.name != ".DS_Store":
        train_input_wav.append(tensor)
    
for tensor in train_output.iterdir():
    if tensor.name != ".DS_Store":
        train_output_wav.append(tensor)

for tensor in test_input.iterdir():
    if tensor.name != ".DS_Store":
        test_input_wav.append(tensor)
    
for tensor in test_output.iterdir():
    if tensor.name != ".DS_Store":
        test_output_wav.append(tensor)
        
print("train lengths:", len(train_input_wav), len(train_output_wav))
print("test lengths:", len(test_input_wav), len(test_output_wav))

train lengths: 45 45
test lengths: 15 15


In [151]:
class Seq2SeqDataset(Dataset):
    """
    Seq2Seq Dataset. Uses torchaudio + mu law to process wav files.
    """

    def __init__(self, input_wavs, output_wavs, transform=None):
        """
        Args:
            input_wavs: list of input wavs from 1st instrument
            output_wavs: list of output wavs from 2nd instrument
        """
        self.input_wavs = input_wavs
        self.output_wavs = output_wavs
        
    def __len__(self):
        return len(self.input_wavs)

    def __getitem__(self, index):
        input = torch.load(self.input_wavs[index])
        output = torch.load(self.output_wavs[index])
        return torch.cat((input[::100], torch.tensor([EOS_token])), 0), torch.cat((output[::100], torch.tensor([EOS_token])), 0)

Create data loaders using Seq2SeqDataset

In [152]:
train_set = Seq2SeqDataset(train_input_wav, train_output_wav)
test_set = Seq2SeqDataset(test_input_wav, test_output_wav)
print("Train set size: " + str(len(train_set)))
print("Test set size: " + str(len(test_set)))

kwargs = {'num_workers': 1, 'pin_memory': True} if device == 'cuda' else {} #needed for using datasets on gpu
train_loader = torch.utils.data.DataLoader(train_set, batch_size = 8, shuffle = True, **kwargs)
test_loader = torch.utils.data.DataLoader(test_set, batch_size = 8, shuffle = True, **kwargs)

Train set size: 45
Test set size: 15


In [153]:
for data, target in train_loader:
    print(data[0].size())

torch.Size([224])
torch.Size([224])
torch.Size([224])
torch.Size([224])
torch.Size([224])
torch.Size([224])


In [94]:
class EncoderRNN(nn.Module):
    def __init__(self, seq_length):
        super(EncoderRNN, self).__init__()
        self.gru = nn.GRU(1, 1, num_layers = 1)

    def forward(self, input, hidden):
        output, hidden = self.gru(input, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(seq_length, 1, 1, device=device)

In [33]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [67]:
teacher_forcing_ratio = 0.5

def trainx(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=200):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        print(input_tensor[ei])
        encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [35]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [62]:
def trainIters(encoder, decoder, train_loader, n_iters, print_every=5, plot_every=2, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        for input, target in train_loader:
            input = torch.flatten(input).view(-1, 1)
            target = torch.flatten(target).view(-1, 1)
            loss = train(input, target, encoder,
                         decoder, encoder_optimizer, decoder_optimizer, criterion)
            print_loss_total += loss
            plot_loss_total += loss

            if iter % print_every == 0:
                print_loss_avg = print_loss_total / print_every
                print_loss_total = 0
                print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                             iter, iter / n_iters * 100, print_loss_avg))

            if iter % plot_every == 0:
                plot_loss_avg = plot_loss_total / plot_every
                plot_losses.append(plot_loss_avg)
                plot_loss_total = 0

    showPlot(plot_losses)

In [63]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [98]:
encoder_optimizer = optim.SGD(encoder.parameters(), lr=0.01)
decoder_optimizer = optim.SGD(decoder.parameters(), lr=0.01)
criterion = nn.NLLLoss()

In [115]:
def train(encoder, decoder, epoch):
    for batch_idx, (data, target) in enumerate(train_loader):
        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()
        
        data = torch.unsqueeze(data, 2)
        target = torch.unsqueeze(target, 2)
        data = data.permute(1, 0, 2)
        target = target.permute(1, 0, 2)
        
        print(data)
        data = data.permute(1, 0, 2)
        data = data.requires_grad_() #set requires_grad to True for training
        output = model(data)
        output = output.permute(1, 0, 2) #original output dimensions are batchSizex1x6 
#         print(output[0].shape, target.shape)
        loss = F.nll_loss(output[0], target) #the loss functions expects a batchSizex10 input
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0: #print training stats
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss))

In [116]:
hidden_size = 200
encoder = EncoderRNN(223).to(device)
decoder = EncoderRNN(223).to(device)
train(encoder, decoder, 10)

tensor([[[122],
         [122],
         [121],
         ...,
         [122],
         [121],
         [122]],

        [[128],
         [128],
         [124],
         ...,
         [128],
         [128],
         [128]],

        [[128],
         [122],
         [128],
         ...,
         [128],
         [128],
         [122]],

        ...,

        [[ 12],
         [ 23],
         [ 18],
         ...,
         [233],
         [  9],
         [ 16]],

        [[ 12],
         [ 66],
         [225],
         ...,
         [ 25],
         [ 18],
         [219]],

        [[ 14],
         [236],
         [237],
         ...,
         [ 12],
         [231],
         [204]]])


RuntimeError: only Tensors of floating point dtype can require gradients